In [35]:
import requests
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import log_loss, roc_auc_score

def fetch_usgs_data(start_date, end_date, min_magnitude=2.5):
    url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
    
    try:
        params = {
            "format": "geojson",
            "starttime": start_date,
            "endtime": end_date,
            "minmagnitude": min_magnitude
        }
        
        response = requests.get(url, params=params, timeout=1000)
        response.raise_for_status()
        
        data = response.json()
        features = data.get("features", [])
        
        records = []
        for feature in features:
            props = feature["properties"]
            geometry = feature["geometry"]["coordinates"]
            records.append({
                "time": pd.to_datetime(props["time"], unit="ms"),
                "latitude": geometry[1],
                "longitude": geometry[0],
                "depth": geometry[2],
                "magnitude": props["mag"],
                "event_occurred": 1  
            })
        
        df = pd.DataFrame(records)
        negative_samples = pd.DataFrame({
            "latitude": np.random.uniform(df['latitude'].min(), df['latitude'].max(), 100),
            "longitude": np.random.uniform(df['longitude'].min(), df['longitude'].max(), 100),
            "depth": np.random.uniform(df['depth'].min(), df['depth'].max(), 100),
            "magnitude": np.zeros(100),
            "event_occurred": 0
        })
        
        df = pd.concat([df, negative_samples], ignore_index=True)
        return df
    
    except requests.RequestException as e:
        print(f"Error fetching USGS data: {e}")
        return pd.DataFrame()

def prepare_classification_data(df, lookback=30, test_size=0.2):
    features = ['latitude', 'longitude', 'depth', 'magnitude']
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df[features])
    target = df['event_occurred'].values
    X, y = [], []
    for i in range(len(scaled_data) - lookback):
        X.append(scaled_data[i:i+lookback])
        y.append(target[i+lookback])
    
    X, y = np.array(X), np.array(y)
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=42
    )
    return X_train, X_test, y_train, y_test, scaler

def build_probability_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(64, activation='relu', input_shape=input_shape, return_sequences=True),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.LSTM(32, activation='relu'),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid') 
    ])
    model.compile(
        optimizer='adam', 
        loss='binary_crossentropy', 
        metrics=['accuracy', tf.keras.metrics.AUC()]
    )
    
    return model

def evaluate_probability_model(model, X_test, y_test):
    y_pred_proba = model.predict(X_test).flatten()
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    metrics = {
        'Log Loss': log_loss(y_test, y_pred_proba),
        'ROC AUC': roc_auc_score(y_test, y_pred_proba),
        'Accuracy': np.mean(y_pred == y_test),
    }
    return metrics

def predict_earthquake_probability(model, input_data, scaler):
    scaled_input = scaler.transform(input_data)
    scaled_input = scaled_input.reshape(1, -1, 4)
    probability = model.predict(scaled_input)[0][0]
    return probability * 100

import os

def main_earthquake_probability_prediction(
    start_date='2023-05-01', 
    end_date='2024-01-01', 
    min_magnitude=2.5, 
    lookback=30
):
    earthquake_data = fetch_usgs_data(start_date, end_date, min_magnitude)
    if earthquake_data.empty:
        print("No earthquake data retrieved.")
        return None

    X_train, X_test, y_train, y_test, scaler = prepare_classification_data(
        earthquake_data, lookback=lookback
    )
    
    model = build_probability_model(input_shape=(lookback, 4))
    model.fit(
        X_train, y_train, 
        validation_split=0.2, 
        epochs=50, 
        batch_size=32, 
        verbose=1
    )
    
    model_save_path = "../src/model/eq_lstm_nn"
    os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
    model.save(f"{model_save_path}.keras")
    print(f"Model saved at: {model_save_path}")
    
    metrics = evaluate_probability_model(model, X_test, y_test)
    print("Model Performance Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")

    example_input = np.array([[120.0, 180.0, 20.0, 7.0]])
    earthquake_probability = predict_earthquake_probability(model, example_input, scaler)
    
    print(f"\nProbability of Earthquake at (180.0, 100.0): {earthquake_probability:.2f}%")
    
    return {
        'model': model, 
        'metrics': metrics, 
        'scaler': scaler
    }

result = main_earthquake_probability_prediction()

Epoch 1/50


/home/hamza/Dev/analytics_and_modeling/eq_forecaster/notebooks/eq_forecast_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


368/368 ━━━━━━━━━━━━━━━━━━━━ 11s 20ms/step - accuracy: 0.9760 - auc_15: 0.7698 - loss: 0.1635 - val_accuracy: 0.9993 - val_auc_15: 0.9469 - val_loss: 0.0061
Epoch 2/50
368/368 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - accuracy: 0.9994 - auc_15: 0.9359 - loss: 0.0067 - val_accuracy: 0.9993 - val_auc_15: 0.9737 - val_loss: 0.0039
Epoch 3/50
368/368 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - accuracy: 0.9996 - auc_15: 0.9550 - loss: 0.0041 - val_accuracy: 0.9997 - val_auc_15: 0.9737 - val_loss: 0.0035
Epoch 4/50
368/368 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - accuracy: 0.9996 - auc_15: 0.9554 - loss: 0.0042 - val_accuracy: 0.9997 - val_auc_15: 0.9737 - val_loss: 0.0028
Epoch 5/50
368/368 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - accuracy: 0.9993 - auc_15: 0.9583 - loss: 0.0332 - val_accuracy: 0.9935 - val_auc_15: 0.9610 - val_loss: 0.0307
Epoch 6/50
368/368 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - accuracy: 0.9973 - auc_15: 0.9457 - loss: 0.0136 - val_accuracy: 0.9997 - val_auc_15: 0.9737 - val_loss: 0.0031
Epoch 7/

/home/hamza/Dev/analytics_and_modeling/eq_forecaster/notebooks/eq_forecast_env/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step

Probability of Earthquake at (180.0, 100.0): 76.66%


In [7]:
import requests
import pandas as pd

url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.geojson"
response = requests.get(url)
val = response.json()

# Extracting properties from each feature
properties = [feature['properties'] for feature in val['features']]

# Creating DataFrame from the extracted properties
df = pd.DataFrame(properties)
print(df.head())


    mag                           place           time        updated    tz  \
0  2.00     89 km S of Kokhanok, Alaska  1733939772592  1733939892880  None   
1  1.32     4 km NNW of The Geysers, CA  1733939713940  1733939903080  None   
2  1.90  22 km NNE of Yerington, Nevada  1733939633669  1733939786011  None   
3  2.47         21 km W of Petrolia, CA  1733939060070  1733939158404  None   
4  1.90      22 km N of Nikiski, Alaska  1733938672630  1733938796673  None   

                                                 url  \
0  https://earthquake.usgs.gov/earthquakes/eventp...   
1  https://earthquake.usgs.gov/earthquakes/eventp...   
2  https://earthquake.usgs.gov/earthquakes/eventp...   
3  https://earthquake.usgs.gov/earthquakes/eventp...   
4  https://earthquake.usgs.gov/earthquakes/eventp...   

                                              detail  felt  cdi  mmi  ...  \
0  https://earthquake.usgs.gov/earthquakes/feed/v...   NaN  NaN  NaN  ...   
1  https://earthquake.usgs.gov/ear